### Importing Important Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
import boto3
import uuid

s3 = boto3.resource('s3')
# Add a unique suffix to the bucket name
bucket_name = f'bankapplication-{uuid.uuid4()}'  # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = 'us-east-2'

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region}
        )
    print(f'S3 bucket "{bucket_name}" created successfully')
except Exception as e:
    print('S3 error:', e)


S3 bucket "bankapplication-ad9d1b99-ffe4-4e93-9189-9fb06f06b4df" created successfully


In [7]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication-ad9d1b99-ffe4-4e93-9189-9fb06f06b4df/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [8]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [9]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [11]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
from sagemaker.inputs import TrainingInput

s3_input_train = TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/train', content_type='csv')


In [13]:
import os
import boto3
from sagemaker.inputs import TrainingInput  # use this instead of s3_input

# Save test.csv locally
pd.concat([
    test_data['y_yes'],
    test_data.drop(['y_no', 'y_yes'], axis=1)
], axis=1).to_csv('test.csv', index=False, header=False)

# Upload to S3
s3 = boto3.Session().resource('s3')
s3.Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Define S3 test input using updated API
s3_input_test = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/test',
    content_type='csv'
)


### Building Models Xgboot- Inbuilt Algorithm

In [14]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [17]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

# Create the estimator with the correct argument name
estimator = Estimator(
    image_uri=container,  # ✅ renamed from image_name
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # GB
    max_run=300,
    max_wait=600,
    use_spot_instances=True,
    output_path=output_path,
    hyperparameters=hyperparameters
)


In [18]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-07-09-02-57-09-026


2025-07-09 02:57:11 Starting - Starting the training job...
2025-07-09 02:57:42 Downloading - Downloading input data...
2025-07-09 02:57:57 Downloading - Downloading the training image...
2025-07-09 02:58:33 Training - Training image download completed. Training in progress...[2025-07-09 02:58:52.098 ip-10-0-155-149.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node trainin

### Deploy Machine Learning Model As Endpoints

In [19]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-07-09-03-01-33-005
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-07-09-03-01-33-005
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-07-09-03-01-33-005


------!

#### Prediction of the Test Data

In [21]:
from sagemaker.serializers import CSVSerializer
import numpy as np

# Prepare test data
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Set content type and serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()

# Make prediction
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')

# Convert to numpy array
predictions_array = np.fromstring(predictions[1:], sep=',')

print(predictions_array.shape)


(12357,)


In [22]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [23]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



#### Deleting The Endpoints

In [24]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-07-09-03-01-33-005


[{'ResponseMetadata': {'RequestId': 'SR402CERRD4HR8NR',
   'HostId': 'OpadWjoAico6VOa5Mi3t8ONf8XkuldxOJjg5ATQo2F94EMpLVXEq+s7jHh6VmObZMbCYaQydVhr+s8nccmJiMU3Z02Po2NnTaD/OsOvQD7Y=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'OpadWjoAico6VOa5Mi3t8ONf8XkuldxOJjg5ATQo2F94EMpLVXEq+s7jHh6VmObZMbCYaQydVhr+s8nccmJiMU3Z02Po2NnTaD/OsOvQD7Y=',
    'x-amz-request-id': 'SR402CERRD4HR8NR',
    'date': 'Wed, 09 Jul 2025 03:07:24 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-09-02-57-09-026/profiler-output/system/incremental/2025070902/1752029940.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-09-02-57-09-026/debug-output/claim.smd'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-09-02-57-09-026/debug-output/collections/00000000